# Lesson 7 & 8 - Training & Inference
- 7강과 8강에서는 모델을 학습하고 추론하는 방법에 대해 알아보았습니다.
- 이번 실습 자료에서는 다양한 Loss, Optimizer, Scheduler를 활용하는 방법을 알아봅니다.
- 또한, Checkpoint, Early Stopping과 같은 학습을 도와주는 Callback 방법을 알아봅니다.
- 그리고 Graident Accumulation 방법을 활용하여 학습을 진행해봅니다.
## 0. Libraries & Configurations
- 시각화에 필요한 라이브러리와 학습에 필요한 설정을 합니다.

In [1]:
# !pip install -U ipywidgets > /dev/null

In [2]:
import random
import os, sys
from importlib import import_module

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

sys.path.append(os.path.abspath('..'))
from dataset import MaskBaseDataset
from model import *

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

In [3]:
# -- parameters
img_root = '/opt/ml/input/data/train/images'  # 학습 이미지 폴더의 경로
label_path = '/opt/ml/input/data/train/train.csv'  # 학습 메타파일의 경로

# model_name = "BaseModel"  # 모델 이름
model_name = "MyModel"  # 모델 이름
use_pretrained = True  # pretrained-model의 사용 여부
freeze_backbone = False  # classifier head 이 외 부분을 업데이트되지 않게 할 것인지 여부

batch_size = 64
num_workers = 4
num_classes = 18

num_epochs = 5  # 학습할 epoch의 수
lr = 1e-4
lr_decay_step = 10

train_log_interval = 20  # logging할 iteration의 주기
name = "02_vgg"  # 결과를 저장하는 폴더의 이름

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## 1. Loss
- Image Classification에 사용되는 다양한 loss 함수들이 존재합니다. 각 loss 함수는 목적이 있고 풀고자 하는 문제에 맞게 적용을 해야합니다.
- Cross Entropy Loss는 두 분포간의 불확실성을 최소화 하는 목적을 가진 분류에 사용되는 일반적인 손실함수입니다.
- Focal Loss는 Imbalanced Data 문제를 해결하기 위한 손실함수입니다. [참고](https://arxiv.org/pdf/1708.02002.pdf)
- Label Smoothing은 학습 데이터의 representation을 더 잘나타내는데 도움을 줍니다. [참고](https://arxiv.org/pdf/1906.02629.pdf)
- F1 Loss는 F1 score 향상을 목적으로 하는 손실함수입니다.

In [4]:
# -- Cross Entropy Loss
class CrossEntropyLoss(nn.Module):
    def __init__(self, weight=None, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [5]:
# -- Focal Loss
# https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/8
class FocalLoss(nn.Module):
    def __init__(self, weight=None,
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )

In [6]:
# -- Label Smoothing Loss
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=3, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [7]:
# -- F1 Loss
# https://gist.github.com/SuperShinyEyes/dcc68a08ff8b615442e3bc6a9b55a354
class F1Loss(nn.Module):
    def __init__(self, classes=3, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [8]:
criterion = CrossEntropyLoss()

## 2. Optimizer
- 파이토치는 코드를 간단히 수정하여 다양한 optimizer를 사용할 수 있습니다.
- 또한 Model의 레이어마다 다른 learning rate를 적용할 수도 있습니다.

In [9]:
# -- model
model_module = getattr(import_module("model"), model_name)  # default: BaseModel
model = model_module(
    num_classes=num_classes
).to(device)
model = torch.nn.DataParallel(model) # 데이터 병렬처리. 다수의 GPU 사용가능하게. 

In [10]:
# -- SGD optimizer
optimizer = SGD(model.parameters(), lr=lr, weight_decay=5e-4)

In [11]:
# -- Adam optimizer
optimizer = Adam(model.parameters(), lr=lr, weight_decay=5e-4)

- 한 모델에 다른 learning rate를 적용시키기 위해 모델의 구조를 살펴봅시다.

In [12]:
list(filter(lambda kv: 'module.fc' in kv[0], model.named_parameters()))

[]

In [13]:
# -- optimizer: Different Learning Rates on different layers

# features 레이어와 classifier 레이어에서 서로 다른 learning rate를 적용하여 optimizer를 정의할 수 있습니다.
params = [v for k, v in list(filter(lambda kv: 'module.fc' not in kv[0], model.named_parameters()))]
fc_params = [v for k, v in list(filter(lambda kv: 'module.fc' in kv[0], model.named_parameters()))]
train_params = [{'params': params, 'lr': lr / 10, 'weight_decay':5e-4},
                {'params': fc_params, 'lr': lr, 'weight_decay':5e-4}]
optimizer = Adam(train_params)

## 3. Scheduler
- Scheduler은 optimizer의 learning rate를 동적으로 변경시키는 기능을 합니다.
- Optimizer과 Scheduler를 적절히 활용하면 모델이 좋은 성능으로 Fitting하는데 도움을 줍니다.

In [14]:
# -- scheduler: StepLR
# 지정된 step마다 learning rate를 감소시킵니다.
scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

In [15]:
# -- scheduler: ReduceLROnPlateau
# 성능이 향상되지 않을 때 learning rate를 줄입니다. patience=10은 10회 동안 성능 향상이 없을 경우입니다.
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=10)

In [16]:
# -- scheduler: CosineAnnealingLR
# CosineAnnealing은 learning rate를 cosine 그래프처럼 변화시킵니다.
scheduler = CosineAnnealingLR(optimizer, T_max=2, eta_min=0.)

## 4. Metric
- Classification 성능을 표현할 때 다양한 평가지표가 있습니다.
- Accuracy: 모델이 정확하게 예측한 객체의 비율
- True Positive(TP): 실제 True인 정답을 True라고 예측 (정답)
- False Positive(FP): 실제 False인 정답을 True라고 예측 (오답)
- False Negative(FN): 실제 True인 정답을 False라고 예측 (오답)
- True Negative(TN): 실제 False인 정답을 False라고 예측 (정답)
- Precision(정밀도): TP / (TP + FP)
- Recall(재현율): TP / (TP + FN)

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]

In [18]:
# -- Accuracy
accuracy_score(y_true, y_pred)

0.3333333333333333

In [19]:
# -- Accuracy
# Normalize를 안하면 맞춘 개수가 표시된다
accuracy_score(y_true, y_pred, normalize=False)

2

In [20]:
# -- Precision
precision = precision_score(y_true, y_pred, average='macro')
precision

0.2222222222222222

In [21]:
# -- Recall
recall = recall_score(y_true, y_pred, average='macro')
recall

0.3333333333333333

In [22]:
# -- f1 score
2 * (precision * recall) / (precision + recall)

0.26666666666666666

In [23]:
# -- f1 score (sklearn)
f1_score(y_true, y_pred, average='macro')

0.26666666666666666

## 5. Training process

In [24]:
# -- dataset
dataset_module = getattr(import_module("dataset"), 'MaskBaseDataset')
dataset = dataset_module(
    data_dir=img_root,
)
num_classes = dataset.num_classes  # 18

# -- augmentation
transform_module = getattr(import_module("dataset"), 'BaseAugmentation')
transform = transform_module(
    resize=[128, 96],
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform)


# -- data_loader
train_set, val_set = dataset.split_dataset()

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False
)

In [38]:
from pathlib import Path

BASE_DIR = Path().resolve().parent.parent 
BASE_DIR

PosixPath('/opt/ml')

In [41]:
pkl_path = BASE_DIR / 'output' / 'all_df.pkl'
pkl_path

PosixPath('/opt/ml/output/all_df.pkl')

In [43]:
df = pd.read_pickle(pkl_path)
df.head()

,id,gender,race,age,path,filepath,filename,mask_type,age_category,category,cat_code,gender_code,mask_code
4,000001,female,Asian,45,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,incorrect_mask.jpg,incorrect,middle,"(female, middle, incorrect)",0,0,0
3,000001,female,Asian,45,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,mask1.jpg,mask,middle,"(female, middle, mask)",1,0,1
6,000001,female,Asian,45,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,mask2.jpg,mask,middle,"(female, middle, mask)",1,0,1
2,000001,female,Asian,45,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,mask3.jpg,mask,middle,"(female, middle, mask)",1,0,1
0,000001,female,Asian,45,000001_female_Asian_45,/opt/ml/input/data/train/images/000001_female_...,mask4.jpg,mask,middle,"(female, middle, mask)",1,0,1


In [24]:
# -- dataset
dataset_module = getattr(import_module("dataset"), 'MaskBaseDataset')
dataset = dataset_module(
    data_dir=img_root,
)
num_classes = dataset.num_classes  # 18

# -- augmentation
transform_module = getattr(import_module("dataset"), 'BaseAugmentation')
transform = transform_module(
    resize=[128, 96],
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform)


# -- data_loader
train_set, val_set = dataset.split_dataset()

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False
)

### 5.1 Callback - Checkpoint, Early Stopping

In [25]:
# -- Callback1: Checkpoint - Accuracy가 높아질 때마다 모델을 저장합니다.
# 학습 코드에서 이어집니다.

# -- Callback2: Early Stopping - 성능이 일정 기간동안 향상이 없을 경우 학습을 종료합니다.
patience = 10
counter = 0
# 학습 코드에서 이어집니다.

### 5.2 Training Method - Gradient Accumulation
- Graident Accumulation은 한 iteration에 파라미터를 업데이트시키는게 아니라, gradient를 여러 iteration 동안 쌓아서 업데이트시킵니다. 한 번에 파라미터를 업데이트시키는 건 noise가 있을 수 있으므로, 여러번 쌓아서 한번에 업데이트 시킴으로써 그러한 문제를 방지하기 위함입니다.

In [26]:
# -- Gradient Accumulation
accumulation_steps = 2
# 학습코드에서 이어집니다.

### 5.3 Training Loop

In [27]:
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

counter = 0
best_val_acc = 0
best_val_loss = np.inf
for epoch in range(num_epochs):
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    for idx, train_batch in enumerate(train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)

        loss.backward()
        
        # -- Gradient Accumulation
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % train_log_interval == 0:
            train_loss = loss_value / train_log_interval
            train_acc = matches / batch_size / train_log_interval
#             current_lr = scheduler.get_last_lr()
            current_lr = scheduler.get_lr()
            print(
                f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )

            loss_value = 0
            matches = 0

    scheduler.step()

    # val loop
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(val_set)
        
        # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print("New best model for val accuracy! saving the model..")
            torch.save(model.state_dict(), f"results/{name}/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            break
        
        
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )

Epoch[0/5](20/237) || training loss 2.639 || training accuracy 25.16% || lr [1e-05, 0.0001]
Epoch[0/5](40/237) || training loss 2.14 || training accuracy 34.38% || lr [1e-05, 0.0001]
Epoch[0/5](60/237) || training loss 1.858 || training accuracy 46.33% || lr [1e-05, 0.0001]
Epoch[0/5](80/237) || training loss 1.683 || training accuracy 48.20% || lr [1e-05, 0.0001]
Epoch[0/5](100/237) || training loss 1.527 || training accuracy 51.33% || lr [1e-05, 0.0001]
Epoch[0/5](120/237) || training loss 1.361 || training accuracy 59.45% || lr [1e-05, 0.0001]
Epoch[0/5](140/237) || training loss 1.277 || training accuracy 64.53% || lr [1e-05, 0.0001]
Epoch[0/5](160/237) || training loss 1.093 || training accuracy 70.31% || lr [1e-05, 0.0001]
Epoch[0/5](180/237) || training loss 0.9215 || training accuracy 74.30% || lr [1e-05, 0.0001]
Epoch[0/5](200/237) || training loss 0.8257 || training accuracy 74.45% || lr [1e-05, 0.0001]
Epoch[0/5](220/237) || training loss 0.7829 || training accuracy 76.33% |

In [28]:
# MyModel

print(inputs.shape)
print(labels.shape)
print(outs.shape)
print(preds.shape)

torch.Size([4, 3, 128, 96])
torch.Size([4])
torch.Size([4, 18])
torch.Size([4])


In [184]:
# BaseModel

print(inputs.shape)
print(labels.shape)
print(outs.shape)
print(preds.shape)

torch.Size([24, 3, 128, 96])
torch.Size([24])
torch.Size([24, 18])
torch.Size([24])


## 6. Reference
- [sumni blog post](https://sumniya.tistory.com/26)

In [29]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [30]:
from pathlib import Path

BASE_DIR = Path('/opt/ml/').resolve()
BASE_DIR

PosixPath('/opt/ml')

In [31]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.

# test_dir = '/mnt/ssd/data/mask_final/public'
test_dir = BASE_DIR / 'input' / 'data' / 'eval'
test_dir

PosixPath('/opt/ml/input/data/eval')

In [32]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [33]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

In [34]:
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_mymodel.csv'), index=False)
print('test inference is done!')

test inference is done!
